In [ ]:
# Characterisation can be done by changing settings to qibolab/runcards/tii5q.yml and examples/tii/diagnostics.yml
# These scripts do not save the characterisation results to the runcard

In [ ]:
from qibolab.paths import script_folder, qibolab_folder
from qibolab import Platform
from qibolab.calibration import utils
from qibolab.calibration.calibration import Calibration as Diagnostics

diagnostics_settings = script_folder / "diagnostics_5q.yml"
runcard = qibolab_folder / 'runcards' / 'tii5q.yml' 

# Create a platform; connect and configure it
platform = Platform('multiqubit', runcard)
platform.connect()
platform.setup()

# create a diagnostics/calibration object
ds = Diagnostics(platform, diagnostics_settings)

# select the qubit to be characterised
qubit = 1

Flux Spectroscopy

In [ ]:
for qubit in range(6):
    for fluxline in range(5):
        ds.run_resonator_spectroscopy_flux(qubit, fluxline)

In [ ]:
for n in range(24, 6*5):
    qubit = n // 5
    fluxline = n % 5
    print(f"scan:{n} qubit: {qubit} fluxline: {fluxline}\n")
    ds.run_resonator_spectroscopy_flux(qubit, fluxline)

In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np 
import os
from qibolab.paths import qibolab_folder
files = []
base_folder = qibolab_folder / 'calibration' / 'data'/ 'quantify' / '20220626' # Replace with date!

for experiment_folder in os.listdir(base_folder):
    file_path = base_folder / experiment_folder / 'dataset.hdf5'
    files.append(str(file_path))

l = len(files)
files.sort()

plt.subplots(figsize=(80,60))

plt.rcParams.update({'font.size': 20})
cols = [f'qubit {q}\nGHz' for q in range(5)]
cols.append('qubit 6 (witness)\nGHz')

rows = [f'flux line {fl}\nmA' for fl in range(5)]

for n, file in enumerate(files):
    f = h5py.File(file)
    xlen = f.attrs['xlen']
    ylen = f.attrs['ylen']
    x = (f['x0'][:xlen] + 20e6) / 1e9
    y = f['x1'][:].reshape(xlen, ylen)[:,0] * 1000
    z = np.flipud(f['y0'][:].reshape(f.attrs['ylen'], f.attrs['xlen'])) * 1e6
    qubit = n // 5
    fluxline = n % 5
    ax = plt.subplot(5,6, qubit+1 + fluxline*(6))
    if qubit == 0:
        ax.set_ylabel(rows[fluxline], size='large')
    if fluxline == 0:
        ax.set_title(cols[qubit], rotation=0, size='large', pad = 20)
    plt.imshow(z, extent=[min(x),max(x),min(y),max(y)], aspect="auto")


plt.subplots_adjust(bottom=0.1, right=0.8, top=0.95)
cax = plt.axes([0.85, 0.1, 0.015, 0.85])
plt.colorbar(cax=cax,label='\u03BCV')
plt.suptitle('Resonator Spectroscopy Flux Matrix', fontsize=40, verticalalignment = 'bottom')
plt.savefig('resonator spectroscopy flux matrix.png', bbox_inches='tight')
plt.show()


### Single Qubit Spectroscopies & Rabi

In [ ]:
qubit = 2

Resonator Spectroscopy

In [ ]:
resonator_freq, avg_voltage, peak_voltage, dataset = ds.run_resonator_spectroscopy(qubit)

In [ ]:
print(f"Average voltage: {int(avg_voltage)}")
print(f"Peak voltage: {int(peak_voltage)}")
print(f"Resonator frequency: {resonator_freq}")
print(f"LO frequency: {resonator_freq - platform.settings['native_gates']['single_qubit'][qubit]['MZ']['frequency']}")

Resonator Punchout

In [ ]:
dataset = ds.run_resonator_punchout(qubit)

Resonator Spectroscopy Flux

In [ ]:
dataset = ds.run_resonator_spectroscopy_flux(qubit, fluxline = 2)

Qubit Spectroscopy

In [ ]:
qubit_freq, avg_voltage, peak_voltage, dataset = ds.run_qubit_spectroscopy(qubit)

In [ ]:
print(f"Average voltage: {int(avg_voltage)}")
print(f"Peak voltage: {int(peak_voltage)}")
print(f"Qubit frequency: {qubit_freq}")
print(f"LO frequency: {qubit_freq - platform.settings['native_gates']['single_qubit'][qubit]['RX']['frequency']}")

Qubit Spectroscopy Flux with LO

In [ ]:
dataset = ds.run_qubit_spectroscopy_flux(qubit, fluxline = 2)

Rabi Oscillations

In [ ]:
pi_pulse_duration, pi_pulse_amplitude, state1_voltage, dataset = ds.run_rabi_pulse_length(qubit)

In [ ]:
print(f"Pi pulse duration: {pi_pulse_duration}")
print(f"Pi pulse amplitude: {pi_pulse_amplitude}")
print(f"Rabi Oscillation pi pulse peak voltage: {state1_voltage}")

Rabi Oscillations with varying pulse gain

In [ ]:
pi_pulse_gain, pi_pulse_amplitude, rabi_oscillations_pi_pulse_peak_voltage, dataset = ds.run_rabi_pulse_gain(qubit)

In [ ]:
print(f"Pi pulse gain: {pi_pulse_gain}")
print(f"Pi pulse amplitude: {pi_pulse_amplitude}")
print(f"Rabi Oscillation pi pulse peak voltage: {state1_voltage}")

T1

In [ ]:
t1, dataset = ds.run_t1(qubit)

Ramsey

In [ ]:
delta_frequency, corrected_qubit_frequency, t2, smooth_dataset, dataset = ds.run_ramsey(qubit)

In [ ]:
print(f"Delta Frequency: {delta_frequency}")
print(f"Corrected qubit frequency: {corrected_qubit_frequency}")
print(f"T2: {t2}")

Ramsey Frequency Detuned

In [ ]:
t2, delta_phys, corrected_qubit_freq, dataset = ds.run_ramsey_frequency_detuned(qubit)

In [ ]:
print(f"Delta Frequency: {delta_phys}")
print(f"Corrected qubit frequency: {corrected_qubit_freq}")
lo_qcm_frequency = corrected_qubit_freq + platform.settings['native_gates']['single_qubit'][qubit]['RX']['frequency']
print(f"Corrected out0_lo_freq: {lo_qcm_frequency}")
print(f"T2: {t2}")

Flipping

In [ ]:
amplitude_delta = ds.run_flipping(qubit)
print(amplitude_delta)

allXY

In [ ]:
results, gateNumber = ds.run_allXY(qubit)

import matplotlib.pyplot as plt

plt.scatter(results, gateNumber)
plt.ylabel('Z projection probability of qubit state |o>')
plt.show()

plt.plot(results)
plt.ylabel('Z projection probability of qubit state |o>')
plt.show()

#TODO: for consistency move all plotting inside the function

Drag Pulse Beta Parameter Tunning

In [ ]:
beta_optimal = ds.run_drag_pulse_tunning(qubit)

## Spectroscopies Loop

In [ ]:
for qubit in range(5):
    resonator_freq, avg_max_voltage, min_ro_voltage, smooth_dataset, dataset = ds.run_resonator_spectroscopy(qubit)


In [ ]:

for qubit in range(5):
    qubit_freq, max_ro_voltage, smooth_dataset, dataset = ds.run_qubit_spectroscopy(qubit)

In [ ]:
while True:
    resonator_freq, avg_min_voltage, max_ro_voltage, smooth_dataset, dataset = ds.run_resonator_spectroscopy(qubit)

In [ ]:
"""
files = []
files.append('D:\\Alvaro\\Projects\\GitHub\\qibolab\\src\\qibolab\\calibration\\data\quantify\\20220625\\20220625-115433-962-82dd8b-matrix3\\dataset.hdf5')
files.append('D:\\Alvaro\\Projects\\GitHub\\qibolab\\src\\qibolab\\calibration\\data\quantify\\20220625\\20220625-124606-847-d03c47-matrix3\\dataset.hdf5')
files.append('D:\\Alvaro\\Projects\\GitHub\\qibolab\\src\\qibolab\\calibration\\data\quantify\\20220625\\20220625-133808-534-7bdec2-matrix3\\dataset.hdf5')
files.append('D:\\Alvaro\\Projects\\GitHub\\qibolab\\src\\qibolab\\calibration\\data\quantify\\20220625\\20220625-143125-212-35e1bb-matrix3\\dataset.hdf5')
files.append('D:\\Alvaro\\Projects\\GitHub\\qibolab\\src\\qibolab\\calibration\\data\quantify\\20220625\\20220625-152641-950-e4bc4b-matrix3\\dataset.hdf5')
"""